In [1]:
#replaced text_cleaning() and extract_text_features()

def RemoveTextPattern(input_txt, pattern):
    import re
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Noise Removal
def scrub_words(text):
    """Basic cleaning of texts."""
    
    import re
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text=text.strip()
    return text

def RemoveCustomizeWords(text):
    #from nltk.corpus import stopwords
    import re
    # lower text 
    text = text.lower()
    #removing stop words
    #text = ' '.join([e_words for e_words in text.split(' ') if e_words not in stopwords.words('english')])
    
    #removing square brackets
    text=re.sub('[.*?]', '', text)
    text=re.sub('+', '', text)
    #removing hyperlink
    text= re.sub('https?://S+|www.S+', '', text)
    #removing puncuation
    text=re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('n' , '', text)
    #remove words containing numbers
    text=re.sub('w*dw*' , '', text)
    #tokenizer
    text = nltk.word_tokenize(text)
    #lemmatizer
    wn = nltk.WordNetLemmatizer()
    text = [wn.lemmatize(word) for word in text]
    text = " ".join(text)
    return text

# Function for expanding contractions
def expand_contractions(text):
    #contractions_dict outside create problem while importing
    contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
    "'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
    "didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
    "hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
    "he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
    "i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
    "isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
    "mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
    "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
    "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
    "she'll've": "she will have","should've": "should have","shouldn't": "should not",
    "shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
    "there'd": "there would","there'd've": "there would have",
    "they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
    "they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
    "what've": "what have","when've": "when have","where'd": "where did",
    "where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
    "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
    "you're": "you are","you've": "you have"}

    
    
'''
TBA:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["content"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["content"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["content"]]
data = data[["content", "Positive", "Negative", "Neutral"]]

positive =' '.join([i for i in data['content'][data['Positive'] > data["Negative"]]])
negative =' '.join([i for i in data['content'][data['Negative'] > data["Positive"]]])

'''



def text_clean_feat_extract(data, text_col=None, cleaning_list=[], cleaning_dict={},
        extract_feat_list=[], extract_feat_dict={}, series_out=True, drop_text_col=True,
        encoding_type=['pert_dist'], add_num_feat=True, drop_single_value_cols=False):
    
    '''
    v1.5 : Apr27,2023 bug fixed of argumnet text_col not needed @ single feat df like series/ TBC
    v1.4 : Mar02,2023 bug fixed, drop_single_value_cols=True
    v1.3 : Feb08,2023 added add_num_feat option/drop single value cols/cat to num
    v1.2 : Dec 05,2022 added drop_text_col/encoding_type
    v1.1 : Nov 29,2022 updated for cleaning_list/cleaning_dict and extract_feat_list/extract_feat_dict
    v1.0 : Nov22,2022 initial ver
     
    assumption:
    https://pypi.org/project/nlp-text-cleaner/
    !pip install nlp-text-cleaner
    
    list: used for setting the flag
    dict: used for to collect external values
    
    cleaning_list=['LowerCase', 'RemovePunct',  
              'RemoveUnicode', 'RemoveLeadTrailWhiteSpaces', 'RemoveDupWhiteSpaces',
              'CorrectGrammar', 'RemoveStopwords', 'ApplyStem', 'ApplyLamma',
              'RemoveHashTags', 'RemoveHyperLinks', 'CleanHtmlCode', 'ReplaceAbbr', 'NoiseRemoval']
              
    TBA >> 'RemoveCustWords', 'ReplaceExtAbbr'
        
    cleaning_dict = {'ExtRemoveStopWords':['a'], 'RemoveCommonWords':10, 'RemoveRareWords':10,
    'RemovePattern':"@[w]*", 'RemoveShortWords':2}
    
    extract_feat_list = ['DetectLang', 'SplitIntoSent', 'SplitIntoWords', 'GetPosTags',
    'StopWordsCount', 'TokensFeat', 'SentiMent', 'POS', 'NamedEntity'
    ]
    #'CharCount', 'WordCount', 'UniqueWordCount', 'UrlCount', 'MeanWordength',
    extract_feat_dict = { 'TokensCountList':['a'], 'TokensPresentList':['a']}
    
    Assumptions:
    only cleaning: series_out=True/False // drop_text_col=True/False
    only feat extraction: series_out=False // if pass directly for model fit, drop_text_col=True
    Both cleaning /feat extraction: try 1 by 1 // drop_text_col=False
    add_num_feat: only feat extraction time it will merge num feat 
    
    Error:
    drop_single_value_cols - ValueError: X has 10 features, but MinMaxScaler is expecting 11 features as input.
    '''
    print('entry text_clean_feat_extract')
    import numpy as np
    import pandas as pd
    from collections import Counter
    import re
    from nlp_text_cleaner import nlp_text_cleaner as cleaner
    
    import string
    import langdetect
    from textblob import TextBlob
    
    import nltk
    from nltk.stem import WordNetLemmatizer
    from nltk import tokenize,ngrams
    from nltk.corpus import stopwords
    nltk.download('vader_lexicon') #download vader from nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer() #creating an object of sentiment intensity analyzer
    
    print('data-typ before chg:', type(data))
    
    print('##1 input format')
    if isinstance(data, pd.DataFrame):
        df = data.copy()
        text_col = text_col
    elif isinstance(data, pd.Series):
        df = pd.DataFrame(data)
        text_col = data.name
    elif (isinstance(data, np.ndarray)) | (isinstance(data, list)):
        df = pd.DataFrame(data)
        text_col = 0
    else:
        pass
    
    print('\n data-typ after chg, should be dataframe:', type(df))
    print(f'\n input data>>\n cleaning_list:{cleaning_list}\n cleaning_dict:{cleaning_dict}\
    \n extract_feat_list:{extract_feat_list} \n extract_feat_dict:{extract_feat_dict}')
    
    print(f'\n text_col:{text_col}, series_out:{series_out}, drop_text_col:{drop_text_col}\
    encoding_type:{encoding_type}, add_num_feat:{add_num_feat}, drop_single_value_cols:{drop_single_value_cols}')
    
    print('execution type chk')
    if cleaning_list or cleaning_dict:
        print('execution for cleaning')
    elif extract_feat_list or extract_feat_dict:
        print('execution for feature extraction')
    elif cleaning_list and cleaning_dict and extract_feat_list and extract_feat_dict:
        print('execution for cleaning/feature extraction')
    else:
        print('execution for nothing')
    
    print('b4 df-cols:', df.columns.tolist())
    text_col = 'text' #bug Apr 27, commented dueto mismatch of text_col
    df = df[[text_col]] #v1.3 only text feat
    print('after df-cols:', df.columns.tolist())
    
    if extract_feat_list:
        print('##2 extract feat w.r.t extract_feat_list flag')
        #if encoding_type:
        #2.1 internal inbuilt functions used for features
        if 'DetectLang' in extract_feat_list:
            df['lang'] = df[text_col].apply(lambda x: cleaner.detect_language(x))
        if 'GetPosTags' in extract_feat_list:   
            pass
            #df['pos_tags'] = df[text_col].apply(lambda x: cleaner.get_pos_tags(x))# it's list of tuples
            
        #2.2 extternal functions/logic used for features
        if 'StopWordsCount' in extract_feat_list:
            print('2.2.1 executing StopWordsCount')
            stop=set(stopwords.words('english'))
            stop_list = list(stop)
            #StopWordsCount = STOPWORDS_list + stop_list #external
            StopWordsCount = stop_list #internal
            df['StopWordsCount'] = df[text_col].apply(lambda x: len([w for w in str(x).lower().split() if w in StopWordsCount]))
            
        if 'TokensFeat' in extract_feat_list:
            print('2.2.2 executing TokenFeat')
            df['char_count'] = df[text_col].apply(len)
            df['word_count'] = df[text_col].apply(lambda x: len(str(x).split()))
            df['unique_word_count'] = df[text_col].apply(lambda x: len(set(str(x).split())))
            df['url_count'] = df[text_col].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
            df['mean_word_length'] = df[text_col].apply(lambda x: round(np.mean([len(w) for w in str(x).split()]),2))
            df['word_density'] = round(df['char_count'] / (df['word_count']+1),2)
            df['punctuation_count'] = df[text_col].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
            df['title_word_count'] = df[text_col].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
            df['upper_case_word_count'] = df[text_col].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
            df['hashtag_count'] = df[text_col].apply(lambda x: len([c for c in str(x) if c == '#']))
            df['mention_count'] = df[text_col].apply(lambda x: len([c for c in str(x) if c == '@']))
            
        if 'SentiMent' in extract_feat_list:
            print('2.2.3 executing SentMent')
            df["sentiment"] = df[text_col].apply(lambda x: round(TextBlob(x).sentiment.polarity),2)
            df['scores']=df[text_col].apply(lambda x: sia.polarity_scores(str(x))) #It's dict, can't be encode
            df['compound'] = df['scores'].apply(lambda score_dict:round(score_dict['compound']),2)
            df['pos'] = df['scores'].apply(lambda pos_dict:round(pos_dict['pos']),2)
            df['neg'] = df['scores'].apply(lambda neg_dict:round(neg_dict['neg']),2)
            #create a new column named type, which indicates whether the review is pos, neg, or neutral.
            df['sentiment_type']=''
            df.loc[df.compound>0,'sentiment_type']='POS'
            df.loc[df.compound==0,'sentiment_type']='NEUTRAL'
            df.loc[df.compound<0,'sentiment_type']='NEG'
            df.drop('scores', axis=1, inplace=True)# it's dict score of pos/neg/neutral
            df['sentiment_type'] = df['sentiment_type'].map({'POS':2, 'NEG':1, 'NEUTRAL':0})
            
        if 'POS' in extract_feat_list:
            print('2.2.4 executing POS')
            pos_family = {
            'noun' : ['NN','NNS','NNP','NNPS'],
            'pron' : ['PRP','PRP$','WP','WP$'],
            'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
            'adj' :  ['JJ','JJR','JJS'],
            'adv' : ['RB','RBR','RBS','WRB']
            }
            
            # function to check and get the part of speech tag count of a words in a given sentence
            def check_pos_tag(x, flag):
                cnt = 0
                try:
                    wiki = textblob.TextBlob(x)
                    for tup in wiki.tags:
                        ppo = list(tup)[1]
                        if ppo in pos_family[flag]:
                            cnt += 1
                except:
                    pass
                return cnt
            df['noun_count'] = df[text_col].apply(lambda x: check_pos_tag(x, 'noun'))
            df['verb_count'] = df[text_col].apply(lambda x: check_pos_tag(x, 'verb'))
            df['adj_count'] = df[text_col].apply(lambda x: check_pos_tag(x, 'adj'))
            df['adv_count'] = df[text_col].apply(lambda x: check_pos_tag(x, 'adv'))
            df['pron_count'] = df[text_col].apply(lambda x: check_pos_tag(x, 'porn'))
            
            #not working / TBT
#             df.assign(
#             noun_count = df[text_col].apply(lambda x: check_pos_tag(x, 'noun')),
#             verb_count = df[text_col].apply(lambda x: check_pos_tag(x, 'verb')),
#             adj_count = df[text_col].apply(lambda x: check_pos_tag(x, 'adj')),
#             adv_count = df[text_col].apply(lambda x: check_pos_tag(x, 'adv')),
#             pron_count = df[text_col].apply(lambda x: check_pos_tag(x, 'pron'))
#                  )
            
        if 'NamedEntity' in extract_feat_list:
            pass
            #Named-Entity Recognition - TBT
        #     import spacy
        #     ner = spacy.load("en_core_web_lg")

        #     df["tags"] = df[text_col].apply(lambda x: [(tag.text, tag.label_) for tag in ner(x).ents]) ## tag text and exctract tags into a list
        #     df["tags"] = df["tags"].apply(lambda x: utils_lst_count(x)) ## count tags

        #     ## extract features
        #     tags_set = []
        #     for lst in df["tags"].tolist():
        #          for dic in lst:
        #                 for k in dic.keys():
        #                     tags_set.append(k[1])
        #     tags_set = list(set(tags_set))
        #     for feature in tags_set:
        #          df["tags_"+feature] = df["tags"].apply(lambda x: utils_ner_features(x, feature))

   
    if extract_feat_dict:
        print('##3 extract feat w.r.t extract_feat_dict external values')
        if 'TokensCountList' in extract_feat_dict:
            print('#3.1 TokensCountList')
            TokensCountList = extract_feat_dict['TokensCountList']
            if (isinstance(TokensCountList, str)): # when it's str
                TokensCountList = list(TokensCountList)
            if TokensCountList:
                df['TokensCount'] = df[text_col].apply(lambda x: len([w for w in str(x).lower().split() if w in TokensCountList]))

        if 'TokensPresentList' in extract_feat_dict:
            print('#3.2 TokensPresentList')
            TokensPresentList = extract_feat_dict['TokensPresentList']
            if (isinstance(TokensPresentList, str)): # when it's str
                TokensPresentList = list(TokensPresentList)
            if TokensPresentList:
                df['TokensPresent'] = df[text_col].str.contains('|'.join(TokensPresentList), case=False)
                df['TokensPresent'] = df['TokensPresent'].astype(int) # convert(True/False-->1/0)
        
    
    if cleaning_list:
        print('##4 clean text w.r.t cleaning_list flag')
        print(f'df-type:{type(df)}, df-len:{len(df)}, df-shp:{df.shape},\
        df-cols:{df.columns.tolist()}, text_col:{text_col}')
        
        #3.1 internal inbuilt functions used for cleaning
        if 'LowerCase' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.lower_case_text(x))
        if 'RemovePunct' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_punctuation(x))
        if 'RemoveUnicode' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_unicode(x))
        if 'RemoveLeadTrailWhiteSpaces' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_leading_trailing_whitespaces(x))
        if 'RemoveDupWhiteSpaces' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_duplicate_whitespaces(x))
        if 'CorrectGrammar' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.correct_grammar(x))
        if 'RemoveStopwords' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_stopwords(x))
        if 'ApplyStem' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.apply_stemming(x))
        if 'ApplyLamma' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.apply_lammatization(x))
        if 'RemoveHashTags' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_hashtags(x))
        if 'RemoveHyperLinks' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.remove_hyperlinks(x))
        if 'CleanHtmlCode' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.clean_html_code(x))
        if 'ReplaceAbbr' in cleaning_list:
            df[text_col] = df[text_col].apply(lambda x: cleaner.replace_contraction(x))
            
        #3.2 external functions used for cleaning
        if 'NoiseRemoval' in cleaning_list:
            df[text_col]=df[text_col].apply(lambda x : " ".join([scrub_words(word) for word in str(x).split()]))
            
        if 'RemoveCustWords' in cleaning_list:#not added in list
            df[text_col] = df[text_col].apply(RemoveCustomizeWords)
            df[text_col] = df[text_col].apply(lambda x :RemoveCustomizeWords(x))
            
        if 'ReplaceExtAbbr' in cleaning_list: #not added in list
            df[text_col]=df[text_col].apply(lambda x:expand_contractions(x))

    else:
        # default method to clean single sentence/
        df[text_col] = df[text_col].apply(lambda x: cleaner.clean_single_sentence(x))
        # default method to get cleaned sentences from a paragraph
        #df[text_col] = df[text_col].apply(lambda x: cleaner.clean_paragraph_to_sentences(x))
        # default method to clean complete paragraph
        #df[text_col] = df[text_col].apply(lambda x: cleaner.clean_paragraph(x))
        
    print('##5 clean text w.r.t cleaning_dict external values')
    #print(f'df-type:{type(df)}, df-len:{len(df)}, df-data:{df}')
    print(f'df-type:{type(df)}, df-shp:{df.shape}')
    if cleaning_dict:
        cnt = Counter()
        for text in df[text_col].values:
            for word in text.split():
                cnt[word] += 1

        if 'RemoveCommonWords' in cleaning_dict:
            print('#5.1 RemoveCommonWords')
            RemoveCommonWords = cleaning_dict['RemoveCommonWords']
            if RemoveCommonWords:
                FREQWORDS = set([w for (w, wc) in cnt.most_common(RemoveCommonWords)])
                df[text_col]=df[text_col].apply(lambda x : " ".join([word for word in str(x).split() if word not in FREQWORDS]))
        if 'RemoveRareWords' in cleaning_dict:
            print('#5.2 RemoveRareWords')
            RemoveRareWords = cleaning_dict['RemoveRareWords']
            if RemoveRareWords:
                RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-RemoveRareWords-1:-1]])
                df[text_col]=df[text_col].apply(lambda x : " ".join([word for word in str(x).split() if word not in RAREWORDS]))

        if 'ExtRemoveStopWords' in cleaning_dict:
            print('#5.3 ExtRemoveStopWords')
            #from nltk.corpus import stopwords
            #stop=set(stopwords.words('english'))
            #internal_STOPWORDS_list = list(stop)
            ExtRemoveStopWords = cleaning_dict['ExtRemoveStopWords']

            if (isinstance(ExtRemoveStopWords, str)): # when it's str
                ExtRemoveStopWords = list(ExtRemoveStopWords)
            #ExtRemoveStopWords = ExtRemoveStopWords + internal_STOPWORDS_list
            if ExtRemoveStopWords:
                df[text_col]=df[text_col].apply(lambda x : " ".join([word for word in str(x).split() if word not in ExtRemoveStopWords]))

        if 'RemovePattern' in cleaning_dict:
            print('#5.4 RemovePattern')
            RemovePattern = cleaning_dict['RemovePattern']
            if RemovePattern:
                df[text_col] = np.vectorize(RemoveTextPattern)(df[text_col], RemovePattern) # function RemoveTextPattern()

        if 'RemoveShortWords' in cleaning_dict: #remove_short_words_length
            print('#5.5 RemoveShortWords')
            RemoveShortWords = cleaning_dict['RemoveShortWords']
            if RemoveShortWords:
                df[text_col] = df[text_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>RemoveShortWords]))

    print('##6 ouput format')
    #print(f'df-type:{type(df)}, df-len:{len(df)}, df-data:{df}')
    print(f'df-type:{type(df)}, df-shp:{df.shape}')
    #print('df-cols:', df.columns.tolist())
    if series_out: # return series / only cleaning
        print('if series_out:', series_out)
        return df[text_col].squeeze()
    else: #testing after clean / cleaning&feat_extract
        print('else series_out:', series_out)
        df['word_count'] = df[text_col].apply(lambda x: len(str(x).split()))
        df['char_count'] = df[text_col].apply(len)
        #print(f'1df-type:{type(df)}, df-len:{len(df)}, df-data:{df}')
        print(f'1df-type:{type(df)}, df-len:{len(df)}, df-shp:{df.shape}')
        
        #change column position for better view Nov15,2022
        first_column = df.pop('word_count')
        df.insert(0, 'word_count', first_column)
        second_column = df.pop('char_count')
        df.insert(1, 'char_count', second_column)
        #print(f'2df-type:{type(df)}, df-len:{len(df)}, df-data:{df}')
        print(f'2df-type:{type(df)}, df-len:{len(df)}, df-shp:{df.shape}')
        
    #print('6.1 df-head-testing1:', df.head())#Mar02,2023
    print('df-cols:', df.columns.tolist())
    
    if drop_text_col:
        print('##7 drop_text_col:', drop_text_col)
        df.drop(text_col, axis=1, inplace=True)
        
    #if add_num_feat and extract_feat_list:#v1.3
    if add_num_feat:#Mar10,2023
        print('##8 merging num features')
        if isinstance(data, pd.DataFrame):
            num_cols_list = list(data.select_dtypes('number').columns)
            if num_cols_list:
                df1 = data[num_cols_list]
                df = pd.concat([df, df1], axis=1)
                
    print('df-cols:', df.columns.tolist())    
    print(f'before drop single value cols>>df-shp:{df.shape}, df-cols:{df.columns.tolist()}')
    if drop_single_value_cols: #v1.4
        df = df[[c for c in list(df) if len(df[c].unique()) > 1]]
    print(f'After drop single value cols>>df-shp:{df.shape}, df-cols:{df.columns.tolist()}')
    #print('8 df-head-testing2:', df.head())#Mar02,2023
    #print(df.info())
    
    
    #convert catg/obj to num
    if extract_feat_list: #v1.4
        cat_cols_list = list(df.select_dtypes(['category', 'object']).columns)
        print('cat_cols_list:', cat_cols_list)

        for col in cat_cols_list:
            print('###9 convert catg/obj to num')
            #map index
            #sorted_indices = df[col].value_counts().index
            #df[col] =  df[col].map(dict(zip(sorted_indices, range(1, len(sorted_indices)+1))))

            #map % distribution
            sorted_pert = df[col].value_counts(normalize=True).to_dict()
            df[col] =  df[col].map(sorted_pert)
    
    #print('9 df-head-testing3:', df.head())#Mar02,2023
    print('df-cols:', df.columns.tolist())
    print('exit text_clean_feat_extract')
    return df

text_cleaning_list=['LowerCase', 'RemovePunct',  
          'RemoveUnicode', 'RemoveLeadTrailWhiteSpaces', 'RemoveDupWhiteSpaces',
          'CorrectGrammar', 'RemoveStopwords', 'ApplyStem', 'ApplyLamma',
          'RemoveHashTags', 'RemoveHyperLinks', 'CleanHtmlCode', 'ReplaceAbbr'] 

text_cleaning_dict = {'ExtRemoveStopWords':['a'], 'RemoveCommonWords':10, 'RemoveRareWords':10,
'RemovePattern':"@[w]*", 'RemoveShortWords':2}

# text_extract_feat_list = ['DetectLang', 'SplitIntoSent', 'SplitIntoWords',
# 'StopWordsCount', 'TokensFeat', 'SentiMent', 'POS', 'NamedEntity']

text_extract_feat_list = ['SplitIntoSent', 'SplitIntoWords',
'StopWordsCount', 'TokensFeat', 'SentiMent', 'POS', 'NamedEntity']

text_extract_feat_dict = {'TokensCountList':['a'], 'TokensPresentList':['a']}

%store text_cleaning_list
%store text_cleaning_dict
%store text_extract_feat_list
%store text_extract_feat_dict

Stored 'text_cleaning_list' (list)
Stored 'text_cleaning_dict' (dict)
Stored 'text_extract_feat_list' (list)
Stored 'text_extract_feat_dict' (dict)


In [2]:
def gensim_embedding(data, fillna=True, vector_size=300, ngram=2, min_count=2, window=5,
                         text_col=None, drop_text_col=True, embedding_type='Word2Vec'):
    '''
    gensim_embedding_w2v replaced by gensim_embedding
    
    v1.3 : Mar18,2023-updated for FastText
    v1.2 : Dec15,2022-updated for Doc2Vec
    v1.1 : Nov11,2022-updated for num cols
    v1.0 : Oct28,2022-initial version
    
    embedding_type: Doc2Vec/Word2Vec/FastText
    '''
    print('entry gensim_embedding')
    from gensim.models import Word2Vec, Doc2Vec
    from gensim.models.doc2vec import Doc2Vec, TaggedDocument
    from gensim.models.fasttext import FastText
    from nltk.tokenize import word_tokenize
    
    
    print('data-typ:', type(data))
   
    if isinstance(data, pd.DataFrame):
        df = data.copy()
        text_col = text_col
        #text_col = 'text' #Nov09,2022
    elif isinstance(data, pd.Series):  #convert i/p series to df for processing Oct20,2022
        df = pd.DataFrame(data)
        text_col = data.name
    elif (isinstance(data, np.ndarray)) | (isinstance(data, list)):
        df = pd.DataFrame(data)
        text_col = 0
    else:
        pass
    
    data1 = df[text_col]
    # Training a word2vec model from the given data set
    if embedding_type=='Word2Vec':
        model = Word2Vec(data1, vector_size=vector_size, min_count=min_count, window=window, sg=ngram, workers=4, epochs=10)
    elif embedding_type=='Doc2Vec':
        tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data1)]
        model = Doc2Vec(tagged_data, vector_size=vector_size, min_count=min_count, window=window, workers=4, epochs=10)
    
    elif embedding_type=='FastText':
        down_sampling = 1e-2
        model = FastText(data1, vector_size=vector_size, window=window, min_count=min_count,
                      sample=down_sampling, workers = 4, sg=ngram, epochs=10)
        
    words_list = list(model.wv.index_to_key)
    words_dict = model.wv.key_to_index
    
    def get_embedding(doc_tokens):
        embeddings = []
        if len(doc_tokens)<1:
            return np.zeros(vector_size)
        else:
            for tok in doc_tokens:
                if tok in model.wv.index_to_key:
                    embeddings.append(model.wv.word_vec(tok))
                else:
                    embeddings.append(np.random.rand(vector_size))
            # mean the vectors of individual words to get the vector of the document
            return np.mean(embeddings, axis=0)
        
    df['vector']=df[text_col].apply(lambda x :get_embedding(x.split()))
    
    #expend vector into diff cols
    df = (df.drop(columns=['vector'])
         .join(df['vector'].apply(pd.Series).add_prefix('vec_'))
         .fillna('') # optional
      )
    if drop_text_col:
        df.drop(text_col, axis=1, inplace=True)
    print('exit gensim_embedding')
    return df

In [3]:
def extract_text_col(df, combine_all_cat_cols=True):
    '''
    ver 1.4 Feb07,2023 added combine_all_cat_cols
    ver 1.3 Feb01,2023 added print for testing
    ver 1.2 Dec09,2022 moved n_token from argument to hardcoded temp to work with function_transformer
    ver 1.1 Oct29,2022 updated combined_text by ' ' instead of '_'
    ver 1.0 Oct12,2022 initial version
    
    check for text col, if more than 1 text col then combine them
    rename text col to 'text'
    return text col name for NLP processing
    '''
    print('Entry extract_text_col')
    from copy import deepcopy
    
    n_token=10 # TBA as arugment 
    
    print('df-cols:', df.columns.tolist())
    df1 = df.copy()
    #df1 = deepcopy(df)
    #df1['text1'] = df1['v2'] # temp for testing
    text_col_list = [col for col in df1.columns if (df1[col].apply(lambda x: len(str(x).split())).max()) > n_token]
    print('text_col_list start:', text_col_list, len(text_col_list))
    
    if len(text_col_list) == 1:
        text_col = text_col_list[0]
    elif len(text_col_list) > 1:
        df1['combined_text'] = df1[text_col_list].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        text_col = 'combined_text'
        df1.drop(text_col_list, inplace=True, axis=1)
    else:
        text_col = None
    print('text_col:', text_col)
    
    df1.rename(columns={text_col: 'text'},  inplace=True)
    print('df1-cols:', df1.columns.tolist())
    
    if combine_all_cat_cols: #v1.4
        cat_cols_list = list(df1.select_dtypes(['category', 'object']).columns)
        if cat_cols_list:
            df1['combined_text_catg'] = df1[df1.columns[0:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
            df1.drop(cat_cols_list, axis=1, inplace=True)
            df1.rename(columns={'combined_text_catg': 'text'},  inplace=True)
            
    print('df1-cols after combine_all_cat_cols:', df1.columns.tolist())
    
    print('Exit extract_text_col')
    return df1



In [4]:
def text_BoW_feature_selection(X, y, analyzer, max_df, min_df, max_features, ngram_range, 
             matrix_type='sparse', BoW_type='CountVect', p_value_limit = 0.95,
            feature_selection='chi2_test', text_col=None, refit=True):
    
    '''
    v1.2 Nov23,2022 bug fixed
    v1.1 Nov18,2022 updated for FunctionTransformer/ renamed function text_BoW_feature_selection_train()
    v1.0: initial verX_valid, 

    BoW_type : Count_vect / TfIdf_vect
    matrix_type : sparse/dense
    refit:  if True then 
    '''
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
    from sklearn.feature_selection import chi2
    import numpy as np
    import pandas as pd
    
    print('entry text_BoW_feature_selection')
    print(f'analyzer:{analyzer}, max_df:{max_df}, min_df:{min_df}, max_features:{max_features},\
    ngram_range:{ngram_range}, BoW_type:{BoW_type}, text_col:{text_col}, refit:{refit}')
    
    data = X.copy()
    print('data-typ b4 chg:', type(data))
    if isinstance(data, pd.DataFrame):
        if len(data.columns)==1: # Nov09,2022
            X = pd.DataFrame(data.values)
            text_col = 0
        else: #when data has num cols Nov09,2022
            text_col = text_col
            X = data.copy()
    elif isinstance(data, pd.Series): #convert i/p series to df for processing Oct20,2022
        X = pd.DataFrame(data)
        text_col = data.name
    elif (isinstance(data, np.ndarray)) | (isinstance(data, list)):
        X = pd.DataFrame(data)
        text_col = 0
    else:
        pass
    print('X-typ after chg:', type(X))
    
    #1 BoW_type selection
    if BoW_type == 'CountVect':
        vect = CountVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range)
    elif BoW_type == 'TfIdfVect':
        vect = TfidfVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range)
    
    train_vect = vect.fit_transform(X[text_col])

    dic_vocabulary = vect.vocabulary_ # dict of ngrams key with count
    feat_list_all = vect.get_feature_names_out() # max_df features from ngrams
    print('#1 BoW_type selection - feat_list_all:', len(feat_list_all))
    
    #2 imp feature selection w.r.r chi_test and p-value
    if feature_selection:
        dtf_features = pd.DataFrame()
        for cat in np.unique(y):
            chi, p = chi2(train_vect, y==cat)
            dtf_features = dtf_features.append(pd.DataFrame({"feature":feat_list_all, "score":1-p, "y":cat}))
            dtf_features = dtf_features.sort_values(["y", "score"], ascending=[True, False])
            dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
        feat_list_selected = dtf_features["feature"].unique().tolist() # imp features from ngrams, selected w.r.t p-value of chi2 test
        print('#2 feature_selection - feat_list_selected:', len(feat_list_selected))
        
    #3 refit the vectorizer on the corpus by giving this new set of words as input, added vocabulary
    if refit:
        if BoW_type == 'CountVect':
            vect = CountVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                                   max_features=max_features, ngram_range= ngram_range, vocabulary=feat_list_selected)
        elif BoW_type == 'TfIdfVect':
            vect = TfidfVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                                   max_features=max_features, ngram_range= ngram_range, vocabulary=feat_list_selected)
        train_vect = vect.fit_transform(X[text_col])
        
        # sparse to dense
        train_vect = train_vect.todense() if matrix_type=='dense' else train_vect

        dic_vocabulary = vect.vocabulary_ # dict of ngrams key with count
        feat_list_final = vect.get_feature_names_out() # max_df features from ngrams
        print(f'#3 refit yes - feat_list_all:{len(feat_list_all)}, feat_list_selected:{len(feat_list_selected)}, feat_list_final:{len(feat_list_final)}')
        print('entry text_BoW_feature_selection')
        return feat_list_final
    else:
        print(f'#3 refit no - feat_list_all:{len(feat_list_all)}, feat_list_selected:{len(feat_list_selected)}')
        print('exit text_BoW_feature_selection')
        return feat_list_selected
    
# from sklearn.preprocessing import FunctionTransformer # sklearn pipeline
# BOW_feat_selection_ft = FunctionTransformer(text_BoW_feature_selection, kw_args= {'analyzer': analyzer,
#     'max_df': max_df, 'min_df': min_df, 'max_features': max_features, 'ngram_range': ngram_range,
#     'BoW_type': BoW_type, 'target': target, 'text_col': text_col, 'refit': refit})

# from imblearn import FunctionSampler # imblearn pipeline
# BOW_feat_selection_fs = FunctionSampler(func=text_BoW_feature_selection, validate=False, kw_args= {'analyzer': analyzer,
#     'max_df': max_df, 'min_df': min_df, 'max_features': max_features, 'ngram_range': ngram_range,
#     'BoW_type': BoW_type, 'target': target, 'text_col': text_col, 'refit': refit})

# %store BOW_feat_selection_ft
# %store BOW_feat_selection_fs

In [5]:
def text_BoW_feature_selection_train(X, y, analyzer, max_df, min_df, max_features, ngram_range, 
             matrix_type='sparse', BoW_type='Count_vect', p_value_limit = 0.95,
            feature_selection='chi2_test'):
    '''
    v1.0: initial verX_valid, 
    
    
    BoW_type : Count_vect / TfIdf_vect
    matrix_type : sparse/dense
    '''
    
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
    from sklearn.feature_selection import chi2
    import numpy as np
    import pandas as pd
    
    if BoW_type == 'Count_vect':
        vect = CountVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range)
    elif BoW_type == 'TfIdf_vect':
        vect = TfidfVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range)
    
    train_vect = vect.fit_transform(X)

    dic_vocabulary = vect.vocabulary_ # dict of ngrams key with count
    X_names = vect.get_feature_names_out() # max_df features from ngrams
    
    #imp feature selection w.r.r chi_test and p-value
    if feature_selection:
        dtf_features = pd.DataFrame()
        for cat in np.unique(y):
            chi, p = chi2(train_vect, y==cat)
            dtf_features = dtf_features.append(pd.DataFrame({"feature":X_names, "score":1-p, "y":cat}))
            dtf_features = dtf_features.sort_values(["y", "score"], ascending=[True, False])
            dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
        X_names = dtf_features["feature"].unique().tolist() # imp features from ngrams, selected w.r.t p-value of chi2 test

    # refit the vectorizer on the corpus by giving this new set of words as input, added vocabulary
    if BoW_type == 'Count_vect':
        vect = CountVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range, vocabulary=X_names)
    elif BoW_type == 'TfIdf_vect':
        vect = TfidfVectorizer(analyzer=analyzer, max_df=max_df, min_df=min_df,
                               max_features=max_features, ngram_range= ngram_range, vocabulary=X_names)
    
    train_vect = vect.fit_transform(X)
    # sparse to dense
    train_vect = train_vect.todense() if matrix_type=='dense' else train_vect
    
    dic_vocabulary = vect.vocabulary_ # dict of ngrams key with count
    X_names = vect.get_feature_names_out() # max_df features from ngrams
    return vect, train_vect
    
    
def text_BoW_feature_selection_test(vect, X_test, matrix_type='sparse'):
    '''
    v1.0 : inital ver
    '''
    test_vect = vect.fit_transform(X_test)
    # sparse to dense
    test_vect = test_vect.todense() if matrix_type=='dense' else test_vect
    return test_vect

def add_vect_text_features(vect, vect_matrix, text_feat_df):
    '''
    v1.0 : initial verison
    1.1 : 
    
    paramters:
    vect: trained vector from count_vect or tfidf_vect
    vect_matrix : count_vect or tfidf_vect output either dense or sparse
    text_feat_df : text_features excluding text/target columns
    
    issues:
    there could be text_col ='text' in vect_matrix so add some unique name to avoid error
    
    '''
    import scipy
    import pandas as pd
    
    # check vect_matrix sparse or dense, for df dense is needed
    if scipy.sparse.issparse(vect_matrix): #sparse --> convert to dense
        vect_df = pd.DataFrame(vect_matrix.todense(), columns=vect.get_feature_names())
    else: #dense
        vect_df = pd.DataFrame(vect_matrix, columns=vect.get_feature_names())
    print('vect_df:', vect_df.shape)
    
    #merge vect with text_feat_df
    vect_df.index = text_feat_df.index # if vect_df does not have original index
    vect_df = pd.concat([vect_df, text_feat_df], axis=1)
    print('vect_df-merged with feat_df:', vect_df.shape)
    return vect_df

In [6]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

#Spelling Correction
#from textblob import TextBlob
#incorrect_text = 'any tezt with for checing'
#textblob = TextBlob(incorrect_text)

In [7]:
# text preprocessing/cleaning TBC
#https://medium.com/@rukshanjayasekara/text-classification-with-fasttext-5cac26ce7bc6

# Handling the retweet(RT) tag
def replace_retweet(tweet, default_replace=""):
    tweet = re.sub('RT\s+', default_replace, tweet)
    return tweet
# Handling user tags(@)
def replace_user(tweet, default_replace="user"):
    tweet = re.sub('\B@\w+', default_replace, tweet)
    return tweet
# Replace emojis with meaningful text
def demojize(tweet):
    tweet = emoji.demojize(tweet)
    return tweet
def replace_url(tweet, default_replace=""):
    tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
    return tweet
# Remove hashtag symbol(#)
def replace_hashtag(tweet, default_replace=""):
    tweet = re.sub('#+', default_replace, tweet)
    return tweet
def to_lowercase(tweet):
    tweet = tweet.lower()
    return tweet
def word_repetition(tweet):
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    return tweet
def punct_repetition(tweet, default_replace=""):
    tweet = re.sub(r'[\?\.\!]+(?=[\?\.\!])', default_replace, tweet)
    return tweet
# Replace contractions with their extended forms
def fix_contractions(tweet):
    tweet = contractions.fix(tweet)
    return tweet

def preprocessor(tweet, verbose=False):
    if verbose: print("Initial tweet: {}".format(tweet))
## Twitter Features
    tweet = replace_retweet(tweet) # replace retweet
    tweet = replace_user(tweet, "") # replace user tag
    tweet = replace_url(tweet) # replace url
    tweet = replace_hashtag(tweet) # replace hashtag
    if verbose: print("Post Twitter processing tweet: {}".format(tweet))
## Word Features
    tweet = to_lowercase(tweet) # lower case
    tweet = fix_contractions(tweet) # replace contractions
    tweet = punct_repetition(tweet) # replace punctuation repetition
    tweet = word_repetition(tweet) # replace word repetition
    tweet = demojize(tweet) # replace emojis
    if verbose: print("Post Word processing tweet: {}".format(tweet))
    return tweet

In [8]:
# tab NN Models

def create_corpus_ListsOfList(df, text_col='text'): #not used yet
    '''
    v1.0: initital draft
    
    returns: lists of list of tokens in text
    
    '''
    from tqdm import tqdm
    from nltk.tokenize import word_tokenize
    
    from nltk.corpus import stopwords
    stop=set(stopwords.words('english'))
    corpus=[]
    for tweet in tqdm(df[text_col]):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

def build_vocab_count_dict(X):
    '''
    v1.0: initital draft
    
    X = df['text']
    returns: token's count dict
    
    '''
    tweets = X.apply(lambda s: s.split()).values      
    vocab_count_dict = {}
    
    for tweet in tweets:
        for word in tweet:
            try:
                vocab_count_dict[word] += 1
            except KeyError:
                vocab_count_dict[word] = 1                
    return vocab_count_dict

def check_embeddings_coverage(embeddings, vocab_count_dict):
    '''
    v1.0: initital draft
    n_covered: Words in the intersection of vocab and embeddings are stored in covered along with their counts.
    n_oov: Words in vocab that don't exist in embeddings are stored in oov along with their counts.
    n_covered and n_oov are total number of counts and they are used for calculating coverage percentages.
    '''
    vocab = vocab_count_dict 
    import operator
    
    covered = {}
    oov = {}    
    n_covered = 0
    n_oov = 0
    
    for word in vocab:
        try:
            covered[word] = embeddings[word]
            n_covered += vocab[word]
        except:
            oov[word] = vocab[word]
            n_oov += vocab[word]
            
    vocab_coverage = len(covered) / len(vocab) # %match of text-vocab with embedding
    text_coverage = (n_covered / (n_covered + n_oov)) # %match of text-vocab with embedding
    print(f'vocab:{len(vocab)}, covered:{len(covered)}, oov:{len(oov)}')
    
    sorted_oov = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return sorted_oov, vocab_coverage, text_coverage, n_covered, n_oov

#corpus = create_corpus(df=tweet, text_col='text')
def text_to_int(X_train_text, X_valid_text, padding_maxlen=100):
    '''
    v1.0 : initial ver
    
    chk:
    #padding_maxlen = X_train_seq.shape[1] #TBC , TBT
    '''
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences

    #1 Tokenize the sentences
    tokenizer = Tokenizer()
    
    #2 preparing vocabulary
    tokenizer.fit_on_texts(list(X_train_text)) # fit on X_train
    #word_index = token.word_index #TBC
    vocab_size=len(tokenizer.word_index) + 1 #+1 for padding, not used dueto high dimension
    
    #3 converting text into integer sequences
    #X_train_seq=tokenizer.texts_to_sequences(corpus) # passed corpus (lists of list of words/sentence)
    X_train_seq  = tokenizer.texts_to_sequences(X_train_text)
    X_valid_seq  = tokenizer.texts_to_sequences(X_valid_text)
    
    #4 padding to prepare sequences of same length
    #X_train_seq  = pad_sequences(X_train_seq, maxlen=padding_maxlen,truncating='post',padding='post') # TBC/Ref
    #padding_maxlen = X_train_seq.shape[1] #TBC , TBT
    X_train_seq  = pad_sequences(X_train_seq, maxlen=padding_maxlen)
    X_valid_seq  = pad_sequences(X_valid_seq, maxlen=padding_maxlen)
    
    word_index=tokenizer.word_index
    print('Number of unique words:', len(word_index))

    return X_train_seq, X_valid_seq, vocab_size, tokenizer

def load_Glove_embedding_vector(File):
    import numpy as np
    print("Loading Glove Model")
    glove_embedding_vector = {}
    with open(File,'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float32)
            glove_embedding_vector[word] = embedding
        f.close()
    #print(f"{len(glove_embedding_vector)} words loaded!")
    return glove_embedding_vector

def embedding_vector_2_matrix(vocab_size, tokenizer, glove_embedding_vector):
    '''
    '''
    from numpy import zeros
    # create a weight matrix for words in training docs
    vector_size = len(list(glove_embedding_vector.values())[0]) # find vector size
    
    embedding_matrix = zeros((vocab_size, vector_size))
    
    for word, i in tokenizer.word_index.items():
        embedding_vector = glove_embedding_vector.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

    
def build_NN_text_class_model(vocab_size, vector_size, X_train_seq, y_train, X_valid_seq, y_valid,
                               trained_weight=None, model_type='learning_embeddings_NN',
                               num_classes = 1, score_metrics = 'accuracy', DropOutRate=0.25, optimizer = 'Adam',
                               n_epochs = 1, batch_size=32, n_neurons= 50, learning_rate=0.01, l2_loss_lambda=0.10,
                               model_save_path='D:\\NLP\\models\\NN_models', test_type='XYZ'):
    '''
    model_type: learning_embeddings_LSTM / trained_embeddings
    X_train_seq.shape[1]
    
    1.0 initial ver
    1.1 updated return and added model_eval_dict
    
    chk
    y_train can be used to find num_classes , binary - 2
    '''
    
    #create model save name
    model_save_pathname = model_save_path+'/'+model_type+'_'+test_type+'.h5'
    
        
    import tensorflow as tf
    
    #set loss and output_activation w.r.t num_classes
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, GlobalMaxPooling2D
    #from keras.layers.core import Dropout, Dense, Flatten, Activation
    from keras import layers, models, optimizers, regularizers
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau
    
    from keras.layers import LSTM, Dense, Dropout, Embedding, Masking
    from keras.utils.np_utils import to_categorical
    
    input_size = X_train_seq.shape[1] # same as padding_maxlen also
    
    if num_classes == 1: #binary it should be 2
        #loss ='sparse_categorical_crossentropy'
        loss = 'binary_crossentropy'
        output_activation = 'sigmoid'
    else:
        loss = 'categorical_crossentropy'
        output_activation = 'softmax'
        
        y_train = to_categorical(y_train, num_classes) #multi classification
        y_valid = to_categorical(y_valid, num_classes)

    # L2 regularization
    l2 = None if l2_loss_lambda is None else regularizers.l2(l2_loss_lambda)
        
    if model_type == 'lrng_embd_LSTM':
        model = Sequential()
        model.add(Embedding(vocab_size, vector_size, input_length=input_size, trainable=True))
        model.add(LSTM(128, return_sequences=True, dropout=0.2))
        model.add(tf.keras.layers.GlobalMaxPooling1D())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation=output_activation, kernel_regularizer=l2))
    elif model_type == 'lrng_embd_NN': #Shallow_NN 
        model = Sequential()
        model.add(Dense(50, input_dim=X_train_seq.shape[1], kernel_initializer="uniform", activation="relu"))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation=output_activation, kernel_regularizer=l2))
    elif model_type == 'lrng_embd_ShlowNN': # same as lrng_embd_NN
        input_layer = layers.Input((input_size, ), sparse=True)
        hidden_layer = layers.Dense(100, activation="relu")(input_layer)
        output_layer = layers.Dense(num_classes, activation=output_activation)(hidden_layer)
        model = models.Model(inputs = input_layer, outputs = output_layer)
        
    #trained models
    # embedding=Embedding(vocab_size, vector_size, embeddings_initializer=Constant(trained_weight), input_length=MAX_LEN, trainable=False)
    elif model_type == 'trnd_embd_LSTM1':
        model = Sequential()
        #model.add(Embedding(input_dim = num_words, output_dim = EMBEDDING_DIM, input_length= X_train_pad.shape[1], weights = [gensim_weight_matrix], trainable = False))
        model.add(Embedding(vocab_size, vector_size, input_length=input_size, trainable=False,  weights=[trained_weight]))
        #model.add(SpatialDropout1D(0.2)) #TBC
        model.add(LSTM(128, return_sequences=True, dropout=0.2))
        model.add(tf.keras.layers.GlobalMaxPooling1D())
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation=output_activation, kernel_regularizer=l2))
    elif model_type == 'trnd_embd_CNN1D':
        input_layer = layers.Input((input_size, ))
        embedding_layer = layers.Embedding(vocab_size, vector_size, trainable=False, weights=[trained_weight])(input_layer)
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer) # Add the convolutional Layer
        pooling_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer) # Add the pooling Layer
        output_layer1 = layers.Dense(50, activation="relu")(pooling_layer) # Add the output Layers
        output_layer1 = layers.Dropout(0.25)(output_layer1) # Add the output Layers
        output_layer2 = layers.Dense(num_classes, activation=output_activation)(output_layer1) #final output Layer
        model = models.Model(inputs=input_layer, outputs=output_layer2)
    elif model_type == 'trnd_embd_LSTM2':
        input_layer = layers.Input((input_size, ))
        embedding_layer = layers.Embedding(vocab_size, vector_size, trainable=False, weights=[trained_weight])(input_layer)
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        lstm_layer = layers.LSTM(100)(embedding_layer)
        output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
        output_layer1 = layers.Dropout(0.25)(output_layer1)
        output_layer2 = layers.Dense(num_classes, activation="sigmoid")(output_layer1)
        model = models.Model(inputs=input_layer, outputs=output_layer2)
    elif model_type == 'trnd_embd_GRU1':
        input_layer = layers.Input((input_size, ))
        embedding_layer = layers.Embedding(vocab_size, vector_size, trainable=False, weights=[trained_weight])(input_layer)
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        gru_layer = layers.GRU(100)(embedding_layer) # GRU layer
        output_layer1 = layers.Dense(50, activation="relu")(gru_layer)
        output_layer1 = layers.Dropout(0.25)(output_layer1)
        output_layer2 = layers.Dense(num_classes, activation="sigmoid")(output_layer1)
        model = models.Model(inputs=input_layer, outputs=output_layer2)
    elif model_type == 'trnd_embd_BidirGRU1':
        input_layer = layers.Input((input_size, ))
        embedding_layer = layers.Embedding(vocab_size, vector_size, trainable=False, weights=[trained_weight])(input_layer)
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)
        output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
        output_layer1 = layers.Dropout(0.25)(output_layer1)
        output_layer2 = layers.Dense(num_classes, activation="sigmoid")(output_layer1)
        model = models.Model(inputs=input_layer, outputs=output_layer2)
    elif model_type == 'trnd_embd_BidirRNN1':
        input_layer = layers.Input((input_size, ))
        embedding_layer = layers.Embedding(vocab_size, vector_size, trainable=False, weights=[trained_weight])(input_layer)
        embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
        rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
        conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)
        pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
        output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
        output_layer1 = layers.Dropout(0.25)(output_layer1)
        output_layer2 = layers.Dense(num_classes, activation="sigmoid")(output_layer1)
        model = models.Model(inputs=input_layer, outputs=output_layer2)
    elif model_type == 'trnd_embd_BidirCuDNNLSTM':
        model = Sequential()
        model.add(Embedding(vocab_size, vector_size, input_length=input_size, trainable=False,  weights=[trained_weight]))
        #model.add(Embedding(input_dim = num_words, output_dim = EMBEDDING_DIM, input_length= X_train_pad.shape[1], weights = [gensim_weight_matrix], trainable = False))
        model.add(Dropout(0.2))
        model.add(layers.Bidirectional(layers.CuDNNLSTM(100,return_sequences=True)))
        model.add(Dropout(0.2))
        model.add(layers.Bidirectional(layers.CuDNNLSTM(200,return_sequences=True)))
        model.add(Dropout(0.2))
        model.add(layers.Bidirectional(layers.CuDNNLSTM(100,return_sequences=False)))
        model.add(Dense(num_classes, activation=output_activation, kernel_regularizer=l2))
        #model.add(Dense(class_num, activation = 'softmax'))

    # compile with adam optimizer & categorical_crossentropy loss function
    #model.add(Dense(num_classes, activation=output_activation, kernel_regularizer=l2))
    model.compile(optimizer=optimizer, loss=loss, metrics=[score_metrics])

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)  
    mc=ModelCheckpoint(filepath = model_save_pathname, monitor='val_acc', mode='max', save_best_only=True, verbose=1)  # model_path to be updated
    model_history = model.fit(np.array(X_train_seq), np.array(y_train), batch_size=batch_size, epochs=n_epochs, 
                        validation_data=(np.array(X_valid_seq), np.array(y_valid)), verbose=1, callbacks=[es, mc])

    history_df = pd.DataFrame(model_history.history)

    # evaluate the model
    train_loss, train_acc = model.evaluate(np.array(X_train_seq), np.array(y_train), verbose=0)
    valid_loss, valid_acc = model.evaluate(np.array(X_valid_seq), np.array(y_valid), verbose=0)
    model_eval_dict = model.evaluate(np.array(X_valid_seq), np.array(y_valid), batch_size=batch_size, return_dict=True)

    #save the model
    model.save(model_save_pathname)

    # predict the labels on validation dataset
    test_pred = model.predict(X_valid_seq)
    #size = X_valid_seq[0] # chk y_valid
    #test_pred=np.round(test_pred).astype(int).reshape(size) # reshape to be chk
    #sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':test_pred})
    #sub.to_csv('submission.csv',index=False)
    
    #predict / inverse_predict
    #test_pred = model.predict(test_X)
    #test_pred = scaler_y.inverse_transform(test_pred)

#         if num_classes == 1:
#             pred = [int(round(p[0])) for p in pred] # binary classification TBC
#         else:
#             pred = pred.argmax(axis=-1) # multi classification

    #return model, history_df, train_loss, train_acc, valid_loss, valid_acc
    return model_eval_dict, test_pred, model, history_df

In [9]:
def reset_index(dataframe):
    dataframe = dataframe.reset_index(inplace = False)
    return dataframe

def category_encode(dataframe):
    global category_cols
    global target_col
    X = dataframe[category_cols]
    y = dataframe[target_col]
    ce_ord = ce.OrdinalEncoder(cols=category_cols)
    dataframe[category_cols] = ce_ord.fit_transform(X, y)
    return dataframe

In [10]:
#common functions

#TBR from common_functions
# def chk_create_update_df_dict(data=None, file_path='D:\\XYZ\\', file_name='report_df_output',
#                               dict_key=None, dict_value=None, save_df=None):
#     import os, sys
#     import pickle
#     from datetime import datetime
#     import pandas as pd
#     '''
#     1.2 : added feat_list/feat_dict
#     1.1 : added df_dict
#     ver 1.0
#     '''

#     # 1 create /upload report_df file if exist or create
#     file_path_name = file_path + file_name

#     if os.path.isfile(file_path_name+'_pkl'):
#         print('1 update report_df/report_dict exist, open for use')
#         infile = open(file_path_name+'_pkl', 'rb')
#         report_df = pickle.load(infile)
#         report_dict = pickle.load(infile)
#         df_dict = pickle.load(infile) # added 1.1
#     else:
#         print('1 create new report_df/report_dict')
#         report_df = pd.DataFrame()
#         report_dict = {}
#         df_dict = {} # added 1.1

#     # 2 update the data, convert return dict into series before appending in df, index will be date
#     report_df = report_df.append(pd.Series(data, name=str(datetime.now())))
#     report_dict[dict_key] = dict_value
#     df_dict[dict_key] = save_df # added 1.1

#     # 3 save as excel file for ref and analysis
#     report_df.to_excel(file_path_name + '.xlsx', index_label='index')

#     # 4 save report_df as pickle for future update
#     outfile = open(file_path_name+'_pkl', 'wb')
#     pickle.dump(report_df, outfile)
#     pickle.dump(report_dict, outfile)
#     pickle.dump(df_dict, outfile) # added 1.1
#     outfile.close()
#     return report_df

# tt_cv_pt TBR by latest function tt_cv_pt_class from model_function
def tt_cv_pt(X, y, X_train, y_train, X_valid, y_valid, X_test, y_test, model, param_grid, cv, train_type='tt',
             scoring=('accuracy', 'precision', 'recall', 'f1', 'roc_auc')):
    from sklearn.model_selection import train_test_split, GridSearchCV, KFold
    from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict
    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
    from sklearn.metrics import brier_score_loss, log_loss
    from numpy import argmax, sqrt
    import numpy as np
    import pandas as pd

    '''
    1.2 TBA random search and HyperOpt
    1.1 updated train_type tt/cv/pt -->'TrainTest', 'CrossValid', 'GridSearch'
    1.0 initial ver
    '''
    # 1 split train/test
    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    # 2 call train_test / cross_val / grid_search
    if train_type == 'TrainTest':
        model.fit(X_train, y_train)
        y_preds = model.predict(X_valid)

        # cm = confusion_matrix(y_valid, y_preds)
        # cr = classification_report(y_valid, y_preds)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_valid, y_valid)
        train_accuracy = accuracy_score(y_train, model.predict(X_train))
        test_accuracy = accuracy_score(y_valid, model.predict(X_valid))
        train_precision = precision_score(y_train, model.predict(X_train))
        test_precision = precision_score(y_valid, model.predict(X_valid))
        train_recall = recall_score(y_train, model.predict(X_train))
        test_recall = recall_score(y_valid, model.predict(X_valid))
        train_f1 = f1_score(y_train, model.predict(X_train))
        test_f1 = f1_score(y_valid, model.predict(X_valid))
        y_actual = y_valid
        trained_model = model #for prediction

        return_dict = {'train_score': train_score, 'test_score': test_score,
                       'train_accuracy': train_accuracy, 'test_accuracy': test_accuracy,
                       'train_precision': train_precision, 'test_precision': test_precision,
                       'train_recall': train_recall, 'test_recall': test_recall,
                       'train_f1': train_f1, 'test_f1': test_f1}

        # calculate y_probs
        if hasattr(model, "predict_proba"):
            y_probs = model.predict_proba(X_valid)[:, 1]
        else:  # use decision function
            y_probs = model.decision_function(X_valid)

    elif train_type == 'CrossValid':
        # score = cross_val_score(pipe, X, y, cv=cv).mean()
        return_dict = cross_validate(model, X, y, scoring=scoring, cv=cv, return_train_score=True)
        y_preds = cross_val_predict(model, X, y, cv=cv)
        y_probs = cross_val_predict(model, X, y, cv=cv, method='predict_proba')[:, 1]  # y_probs

        return_dict = dict((k, v.mean()) for k, v in return_dict.items())  # avg of each array items
        y_actual = y
        trained_model = model #for prediction

    elif train_type == 'GridSearch':
        estimator = GridSearchCV(model, param_grid=param_grid, cv=cv)
        # estimator.fit(X, y)
        estimator.fit(X_train, y_train)
        best_score = estimator.best_score_
        best_params = estimator.best_params_
        best_estimator = estimator.best_estimator_
        cv_results = estimator.cv_results_
        y_preds = best_estimator.predict(X_valid)
        y_probs = best_estimator.predict_proba(X_valid)[:, 1]  # y_probs
        gs_result_df = pd.DataFrame(cv_results)

        train_accuracy = accuracy_score(y_train, best_estimator.predict(X_train))
        test_accuracy = accuracy_score(y_valid, best_estimator.predict(X_valid))
        train_recall = recall_score(y_train, best_estimator.predict(X_train))
        test_recall = recall_score(y_valid, best_estimator.predict(X_valid))
        train_precision = precision_score(y_train, best_estimator.predict(X_train))
        test_precision = precision_score(y_valid, best_estimator.predict(X_valid))
        train_f1 = f1_score(y_train, best_estimator.predict(X_train))
        test_f1 = f1_score(y_valid, best_estimator.predict(X_valid))
        y_actual = y_valid
        trained_model = best_estimator #for prediction

        return_dict = {'best_score': best_score, 'best_params': best_params,
                       'train_accuracy': train_accuracy, 'test_accuracy': test_accuracy,
                       'train_precision': train_precision, 'test_precision': test_precision,
                       'train_recall': train_recall, 'test_recall': test_recall,
                       'train_f1': train_f1, 'test_f1': test_f1}

    # 3 calculate common metrics as per probs
    loss_log = log_loss(y_actual, y_probs)
    #brier_loss = brier_score_loss(y_actual, y_probs)
    auc_roc_score = roc_auc_score(y_actual, y_probs)
    model_precision, model_recall, thresold_pr = precision_recall_curve(y_actual, y_probs)
    auc_prc_score = auc(model_recall, model_precision)

    # 4 calculate best_threshold using roc curves
    fpr, tpr, thresholds = roc_curve(y_actual, y_preds)
    gmeans = sqrt(tpr * (1 - fpr))  # calculate the g-mean for each threshold
    ix = argmax(gmeans)  # locate the index of the largest g-mean
    best_threshold = thresholds[ix]
    best_gmean = gmeans[ix]

    common_metrics_dict = {'loss_log': loss_log, #'brier_loss': brier_loss,
                           'auc_roc_score': auc_roc_score,
                           'best_threshold': best_threshold, 'best_gmean': best_gmean, 'train_type': train_type,
                           'auc_prc_score': auc_prc_score, 'trained_model': trained_model}
    return_dict.update(common_metrics_dict)

    # 5 calculate common metrics as per preds & confusion metrics
    cm = confusion_matrix(y_actual, y_preds)
    TN, FP, FN, TP = confusion_matrix(y_actual, y_preds).ravel()
    cr = classification_report(y_actual, y_preds)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP / (TP + FN) if (TP + FN) else 0
    # Specificity or true negative rate
    TNR = TN / (TN + FP) if (TN + FP) else 0
    # Precision or positive predictive value
    PPV = TP / (TP + FP) if (TP + FP) else 0
    # Negative predictive value
    NPV = TN / (TN + FN) if (TN + FN) else 0
    # Fall out or false positive rate
    FPR = FP / (FP + TN) if (FP + TN) else 0
    # False negative rate
    FNR = FN / (FN + TP) if (FN + TP) else 0
    # False discovery rate
    FDR = FP / (TP + FP) if (TP + FP) else 0
    # Overall accuracy
    ACC = (TN + TP) / (TP + FP + TN + FN) if (TP + FP + TN + FN) else 0

    confusion_metrics_dict = {'cm': cm, 'cr': cr, 'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
                              'PPV': PPV, 'NPV': NPV, 'FDR': FDR, 'TPR': TPR, 'FPR': FPR, 'TNR': TNR, 'FNR': FNR,
                              'ACC': ACC}

    return_dict.update(confusion_metrics_dict)
    return return_dict, y_preds, y_probs, y_actual, trained_model


def classification_error_score_metrics(y_actual, y_preds, y_probs, model_name='LSTM', test_type='XYZ',
                                       n_features=1, plot=False, plot_path=None):
    '''
    ver 1.0 intial ver
    ver 1.1 : added mape_mean
    '''

    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
    from sklearn.metrics import brier_score_loss, log_loss
    from numpy import argmax, sqrt
    import numpy as np
    import pandas as pd
    
    # 1 calculate common metrics as per preds
    if y_preds is not None:
        acc = accuracy_score(y_actual, y_preds)
        precision = precision_score(y_actual, y_preds)
        recall = recall_score(y_actual, y_preds)
        f1 = f1_score(y_actual, y_preds)
        
        # 1.1 calculate best_threshold using roc curves
        fpr, tpr, thresholds = roc_curve(y_actual, y_preds)
        gmeans = sqrt(tpr * (1 - fpr))  # calculate the g-mean for each threshold
        ix = argmax(gmeans)  # locate the index of the largest g-mean
        best_threshold = thresholds[ix]
        best_gmean = gmeans[ix]
        
        preds_metrics_dict = {'acc': acc, 'precision': precision, 'recall': recall, 'f1': f1,
                   'best_threshold': best_threshold, 'best_gmean': best_gmean, 'auc_prc_score': auc_prc_score,  
                   'test_type': test_type, 'model_name': model_name}
        
        # 1.2 calculate common metrics as per preds & confusion metrics
        cm = confusion_matrix(y_actual, y_preds)
        TN, FP, FN, TP = confusion_matrix(y_actual, y_preds).ravel()
        cr = classification_report(y_actual, y_preds)
        
        #1.3 Sensitivity, hit rate, recall, or true positive rate
        TPR = TP / (TP + FN) if (TP + FN) else 0
        # Specificity or true negative rate
        TNR = TN / (TN + FP) if (TN + FP) else 0
        # Precision or positive predictive value
        PPV = TP / (TP + FP) if (TP + FP) else 0
        # Negative predictive value
        NPV = TN / (TN + FN) if (TN + FN) else 0
        # Fall out or false positive rate
        FPR = FP / (FP + TN) if (FP + TN) else 0
        # False negative rate
        FNR = FN / (FN + TP) if (FN + TP) else 0
        # False discovery rate
        FDR = FP / (TP + FP) if (TP + FP) else 0
        # Overall accuracy
        ACC = (TN + TP) / (TP + FP + TN + FN) if (TP + FP + TN + FN) else 0

        confusion_metrics_dict = {'cm': cm, 'cr': cr, 'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
                              'PPV': PPV, 'NPV': NPV, 'FDR': FDR, 'TPR': TPR, 'FPR': FPR, 'TNR': TNR, 'FNR': FNR,
                              'ACC': ACC}
        preds_metrics_dict.update(confusion_metrics_dict)
        
    # 2 calculate common metrics as per probs
    if y_probs is not None:
        loss_log = log_loss(y_actual, y_probs)
        #brier_loss = brier_score_loss(y_actual, y_probs)
        auc_roc_score = roc_auc_score(y_actual, y_probs)
        model_precision, model_recall, thresold_pr = precision_recall_curve(y_actual, y_probs)
        auc_prc_score = auc(model_recall, model_precision)
        
        probs_metrics_dict = {'loss_log': loss_log, #'brier_loss': brier_loss,
                           'auc_roc_score': auc_roc_score, 'auc_prc_score': auc_prc_score}
        
        preds_metrics_dict.update(probs_metrics_dict)
    return preds_metrics_dict

In [11]:
def spacy_textacy_entity_extraction_knowlege_graph(text_data, ent_type = "DATE", ent_plot_filter = None, plot_save_path=None):

    '''
    1.0 May03,2023 initial version

    plot_save_path: if not none then it will create Network Graph
    ent_type = NUMERIC, PERSON, ORG, LOC, NUMERIC, DATE
    '''

    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt

    import spacy 
    import textacy  #0.12.0
    import networkx as nx  #3.0 (also pygraphviz==1.10)
    import dateparser #1.1.7
    

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text_data)

    #3 from text to a list of sentences
    lst_docs = [sent for sent in doc.sents]

    dic_sub_vrb_obj = {"id":[], "text":[], "sub":[], "relation":[], "object":[]}
    dic_ent = {"id":[], "text":[], ent_type:[]}
    dic_all_ent = {"id":[], "text":[], 'all_ent':[]}
    dic_all_noun = {"id":[], "text":[], 'noun':[]}

    for n, sentence in enumerate(lst_docs):
        #print(f'\n sentence no:{n}, sentence:{sentence}')
        lst_sub_vrb_obj = list(textacy.extract.subject_verb_object_triples(sentence))
        lst_ent = list(textacy.extract.entities(sentence, include_types={ent_type}))
        lst_all_ent = list(textacy.extract.entities(sentence))
        lst_all_noun = list(textacy.extract.noun_chunks(sentence))
        list_all_tokens = list(textacy.extract.words(sentence))

        #1 lst_ent
        if len(lst_ent) > 0:
            for attr in lst_ent:
                dic_ent["id"].append(n)
                dic_ent["text"].append(sentence.text)
                dic_ent[ent_type].append(str(attr))
        else:
            dic_ent["id"].append(n)
            dic_ent["text"].append(sentence.text)
            dic_ent[ent_type].append(np.nan)

        dtf_ent = pd.DataFrame(dic_ent)

        #2 lst_all_ent
        if len(lst_all_ent) > 0:
            for attr1 in lst_all_ent:
                dic_all_ent["id"].append(n)
                dic_all_ent["text"].append(sentence.text)
                dic_all_ent['all_ent'].append(str(attr1))
        else:
            dic_all_ent["id"].append(n)
            dic_all_ent["text"].append(sentence.text)
            dic_all_ent['all_ent'].append(np.nan)

        dtf_all_ent = pd.DataFrame(dic_all_ent)
        dtf_all_ent = dtf_all_ent.groupby('id').agg(lambda x: list(x)).reset_index()

        #3 lst_sub_vrb_obj
        if len(lst_sub_vrb_obj) > 0:
            for m, sent in enumerate(lst_sub_vrb_obj): #one sentence can have more than 1 one sub/verb/obj
                subj = "_".join(map(str, sent.subject))
                obj  = "_".join(map(str, sent.object))
                relation = "_".join(map(str, sent.verb))
                dic_sub_vrb_obj["id"].append(n)
                dic_sub_vrb_obj["text"].append(sentence.text)
                dic_sub_vrb_obj["sub"].append(subj)
                dic_sub_vrb_obj["object"].append(obj)
                dic_sub_vrb_obj["relation"].append(relation)
        else:
            dic_sub_vrb_obj["id"].append(n)
            dic_sub_vrb_obj["text"].append(sentence.text)
            dic_sub_vrb_obj["sub"].append(np.nan)
            dic_sub_vrb_obj["object"].append(np.nan)
            dic_sub_vrb_obj["relation"].append(np.nan)

        dtf_sub_vrb_obj = pd.DataFrame(dic_sub_vrb_obj)

        #4 lst_all_noun
        if len(lst_all_noun) > 0:
            for attr1 in lst_all_noun:
                dic_all_noun["id"].append(n)
                dic_all_noun["text"].append(sentence.text)
                dic_all_noun['noun'].append(str(attr1))
        else:
            dic_all_noun["id"].append(n)
            dic_all_noun["text"].append(sentence.text)
            dic_all_noun['all_ent'].append(np.nan)

        dtf_all_noun = pd.DataFrame(dic_all_noun)
        dtf_all_noun = dtf_all_noun.groupby('id').agg(lambda x: list(x)).reset_index()

    # merge the all files
    dtf_final = dtf_sub_vrb_obj.merge(dtf_ent[[ent_type, 'id']], how="left", on="id")
    dtf_final = dtf_final.merge(dtf_all_ent[['all_ent', 'id']], how="left", on="id")
    dtf_final = dtf_final.merge(dtf_all_noun[['noun', 'id']], how="left", on="id")

    if plot_save_path:
        if ent_plot_filter:
            dtf_final = dtf_final[(dtf_final["sub"]==ent_plot_filter) | (dtf_final["object"]==ent_plot_filter)]

        ## create full graph
        G = nx.from_pandas_edgelist(dtf_final, source="sub", target="object", 
                                    edge_attr="relation", create_using=nx.DiGraph())
        ## plot
        plt.figure(figsize=(15,10))

        pos = nx.spring_layout(G, k=1)
        node_color = "skyblue"
        edge_color = "black"

        nx.draw(G, pos=pos, with_labels=True, node_color=node_color, 
                edge_color=edge_color, cmap=plt.cm.Dark2, 
                node_size=2000, connectionstyle='arc3,rad=0.1')

        nx.draw_networkx_edge_labels(G, pos=pos, label_pos=0.5, 
                                 edge_labels=nx.get_edge_attributes(G,'relation'),
                                 font_size=12, font_color='black', alpha=0.6)
        plt.show()
        plt.savefig(plot_save_path+'EntityKnowledgeGraph.png')

    return dtf_final

# import pandas as pd
# data_path = 'D:\\projects\\CAI\\'
# plot_save_path = 'D:\\projects\\CAI\\'

# file_name = 'User_Journey.xlsx'
# df = pd.read_excel(data_path+file_name, sheet_name='Chat_dummy')
# df['chat'] = df['chat'].astype('object')
# text_data = ','.join([str(i) for i in df['chat'].values.tolist()])

# df_entity = spacy_textacy_entity_extraction_knowlege_graph(text_data, ent_type = "DATE", ent_plot_filter = None, plot_save_path=plot_save_path)
# df_entity.head()

In [12]:
# https://www.analyticsvidhya.com/blog/2022/09/real-or-not-disaster-tweets-classification-with-roberta/

#1. De-abbreviation
def deabbreviate(text):
    import re
    text = text.upper()
    text = re.sub(r'bAFAIKb', ' As Far As I Know ',text)
    text = re.sub(r'bAFKb', ' From Keyboard ',text)
    text = re.sub(r'bASAPb', ' As Soon As Possible ',text)
    text = re.sub(r'bATKb', ' At The Keyboard ',text)
    text = re.sub(r'bA3b', ' Anytime, Anywhere, Anyplace ',text)
    text = re.sub(r'bBAKb', ' Back At Keyboard ',text)
    text = re.sub(r'bBBLb', ' Be Back Later ',text)
    text = re.sub(r'bBBSb', ' Be Back Soon ',text)
    text = re.sub(r'bBFNb', ' Bye For Now ',text)
    text = re.sub(r'bBRBb', ' Be Right Back ',text)
    text = re.sub(r'bBRTb', ' Be Right There ',text)
    text = re.sub(r'bBTWb', ' By The Way ',text)
    text = re.sub(r'bB4b', ' Before ',text)
    text = re.sub(r'bB4Nb', ' Bye For Now ',text)
    text = re.sub(r'bCUb', ' See You ',text)
    text = re.sub(r'bCUL8Rb', ' See You Later ',text)
    text = re.sub(r'bCYAb', ' See You ',text)
    text = re.sub(r'bFAQb', ' Frequently Asked Questions ',text)
    text = re.sub(r'bFYIb', ' For Your Information ',text)
    text = re.sub(r'bGNb', ' Good Night ',text)
    text = re.sub(r'bGR8b', ' Great ',text)
    text = re.sub(r'bICb', ' I See ',text)
    text = re.sub(r'bLOLb', ' Laughing Out Loud ',text)
    text = re.sub(r'bL8Rb', ' Later ',text)
    text = re.sub(r'bM8b', ' Mate ',text)
    text = re.sub(r'bTHXb', ' Thank You ',text)
    text = re.sub(r'bTTFNb', ' BYE ',text)
    text = re.sub(r'bTTFNb', ' BYE ',text)
    text = re.sub(r'bUb', ' You ',text)
    text = re.sub(r'bU2b', ' You TOO ',text)
    text = re.sub(r'bWTFb', ' What The Heck ',text)
    text = re.sub(r'bW8b', ' Wait ',text)
    text = re.sub(r'bFAVb', ' Favourite ',text)
    text = re.sub(r'bHWYb'," highway ",text)
    text = re.sub(r'bPPLb'," people ",text)
    text = re.sub(r'bGVb'," give ",text)
    text = re.sub(r'bWANNAb'," want to ",text)
    text = text.lower()
    return text

#2. Filter Ascii
def return_ascii(text):
    ret_str = ""
    for char in list(text):
        if char.isascii():
            ret_str += char
    return ret_str

#3. General Preprocessing
punctuations = '''!"#$%&()'*+,-./:;?@[]^_`{|}~'''

def preprocess(text):
    import re
    #1.Removing AM and PM
    text = re.sub(r'b[AP]{1}Mb'," ",text)

    #2.Lowercasing 
    text = text.lower()

    #3.removing mentions
    text =re.sub(r'@[^ ]+',' ',text)

    #4.removing urls
    text = re.sub(r'https*://t.co/w+',' ',text)
    text = re.sub(r'https*://[^ ]+',' ',text)

    #5.De-abbreviating
    text = deabbreviate(text)

    #6.cleaning mscl
    text = re.sub(r"åê"," ",text)
    text = re.sub(r'&[^ ]+'," ",text)
    text = re.sub(r'b[a-z]+-[0-9]+b'," ",text)
    text = re.sub(r'bd+[a-z]+d+b'," ",text)
    text = re.sub(r'brtb'," ",text)
    text = re.sub(r'bfavb'," ",text)
    text = re.sub(r'b[h]{1}a[ha]+b',"haa",text)
    text = re.sub(r'b[a-z]+d+[a-z]+b'," ",text)
    text = re.sub(r'<[^]+>',' ',text)

    #7.removing months
    text = re.sub(r" jan | feb | mar | apr | may | jun | jul | aug | sep | oct | nov | dec ",' ',text)

    #8.decontracting
    text = re.sub(r"aren't",'are not',text)
    text = re.sub(r"won't",' will not ',text)
    text = re.sub(r"bi'mb",' I am ',text)
    text = re.sub(r"bi'db",' I would ',text)
    text = re.sub(r"bit'sb",' it is ',text)
    text = re.sub(r"bthat'sb",' that is ',text)
    text = re.sub(r"bcan'tb",' can not ',text)
    text = re.sub(r"bi'veb",' I have ',text)
    text = re.sub(r"bthere'sb",' there is ',text)
    text = re.sub(r"bdidn'tb",' did not ',text)
    text = re.sub(r"bcouldn'tb",' could not ',text)
    text = re.sub(r"bisn'tb",' is not ',text)
    text = re.sub(r"bwe'reb", ' we are ',text)
    text = re.sub(r"bthey'reb",' they are ',text)
    text = re.sub(r"bdon'tb",' do not ',text)
    text = re.sub(r"blet'sb",' let us ',text)
    text = re.sub(r"bli'lb",' little ',text)
    text = re.sub(r"bshe'sb",' she is ',text)
    text = re.sub(r"bhe'sb",' he is ',text)
    text = re.sub(r"bhow'reb",' How are ',text)
    text = re.sub(r"wasn't",' was not ',text)
    text = re.sub(r"bwhat'sb",' what is ',text)
    text = re.sub(r"bhe'llb",' he will ',text)
    text = re.sub(r"bi'llb",' i will ',text)
    text = re.sub(r"bshe'llb",' she will ',text)
    text = re.sub(r"byou'llb",' you will ',text)
    text = re.sub(r"byou'reb",' you are ',text)
    text = re.sub(r"bwe'veb",' we have ',text)
    text = re.sub(r"byou'veb",' you have ',text)
    text = re.sub(r"bthey'veb",' they have ',text)

    #9.removing expression
    text =re.sub(r'A[^ ]+:',' ',text)
    text =re.sub(r'A[^ ]+ [^ ]+:',' ',text)
    text =re.sub(r'A[^ ]+ [^ ]+ [^ ]+:',' ',text)

    #10.Removing the punctuations
    for p in punctuations:
        text = text.replace(p,"   ")

    #11.removing certain patters
    text = re.sub(r'lo+l',"laughing out loud",text)
    text = re.sub(r'coo+l',"cool",text)
    text = re.sub(r'go+a+l+','goal',text)
    text = re.sub(r'so+',"so",text)
    text = re.sub(r'bo+h+o*b','oh',text)

    #12.Removing the digits
    text = re.sub(r'd+'," ",text)

    #13.New line as space
    text = re.sub(r'n'," ",text)

    #14.Removing extra spaces
    text = re.sub(r'[ ]+'," ",text)

    #15.Stripping the end parts
    text = text.strip()

    #16.removing the accents
    text = return_ascii(text)

    #17.removing certain patterns after getting only the ascii characters
    text = re.sub(r"bcantb",' can not ',text)
    text = re.sub(r"bwontb",' will not ',text)
    text = re.sub(r"bimb",' I am ',text)
    text = re.sub(r"bdidntb",' did not ',text)
    text = re.sub(r"bcouldntb",' could not ',text)
    text = re.sub(r"bisntb",' is not ',text)
    text = re.sub(r"bdontb",' do not ',text)
    text = re.sub(r"blilb",' little ',text)
    text = re.sub(r"balilb",' a little ',text)
    text = re.sub(r"view and download video",' ',text)
    text = re.sub(r"bviaZ",' ',text)

    #18.Removing words with lengths less than 2
    text = [ele for ele in text.split(" ") if len(ele) > 1 ]
    text = " ".join(text)

    return text

In [13]:
#https://www.analyticsvidhya.com/blog/2022/09/understanding-word-embeddings-and-building-your-first-rnn-model/
# clean the text data using regex and data cleaning function
def datacleaning(text):
    whitespace = re.compile(r"s+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = user.sub('', text)
    text = re.sub(r"[[^()]*]","", text)
    text = re.sub("d+", "", text)
    text = re.sub(r'[^ws]','',text)
    text = re.sub(r"(?:@S*|#S*|http(?=.*://)S*)", "", text)
    text = text.lower()
    
    # removing stop-words
    text = [word for word in text.split() if word not in list(STOPWORDS)]
    
    # word lemmatization
    sentence = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        sentence.append(lemmatizer.lemmatize(word,'v'))
        
    return ' '.join(sentence)

In [14]:
def preprocessing(text):
    words = regexp_tokenize(text, pattern='w+|$[d.]+|S+')
    tokens = [w for w in words if w.lower() not in string.punctuation]
    stopw = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopw]
    tokens = [word for word in tokens if len(word)>=3]
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens] 
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text
#df['text']=df['text'].apply(preprocessing)

In [15]:
#https://towardsdatascience.com/the-triune-pipeline-for-three-major-transformers-in-nlp-18c14e20530
# custom linguistic features

class SegmentFeaturizer:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_md")
        self.future_words = ["tomorrow", "future", "futures"]
    
    @staticmethod
    def count_pronouns(doc):
        segment = doc.text.lower().split()
        counter = {"1sg": 0, "1pl": 0}
        for pronoun in FIRST_SINGULAR:
            counter["1sg"] += segment.count(pronoun)
        for pronoun in FIRST_PLURAL:
            counter["1pl"] += segment.count(pronoun)
        return counter

    def get_n_future_oriented_words(self, doc):
        will_aux = [t for t in doc if t.tag_ == "MD" and t.lower_ in {"will", "wo", "shall", "sha"}]
        going_to = [t for t in doc if t.dep_ == "xcomp" and t.head.lemma_ == "go"]
        other_future_words = [t for t in doc if t.lower_ in self.future_words]
        return len(will_aux) + len(going_to) + len(other_future_words)

    @staticmethod
    def get_n_word_mentions(doc):
        segment = doc.text
        word_mentions_dict = {
            "n_trump_mentions": segment.count("Trump"),
            "n_other_candidate_mentions": sum(
                [segment.count(c.title()) for c in CANDIDATE_NAMES]
            ),
        }
        return word_mentions_dict

    @staticmethod
    def get_n_words_before_main_verb(doc):
        numbers = [0]
        for sent in doc.sents:
            main = [t for t in sent if t.dep_ == "ROOT"][0]
            if main.pos_ == "VERB":
                dist_to_init = main.i - sent[0].i
                numbers.append(dist_to_init)
        return np.mean(numbers)

    @staticmethod
    def get_n_complex_clauses(doc):
        embedded_elements_count = []
        for sent in doc.sents:
            n_embedded = len(
                [t for t in sent if t.dep_ in {"ccomp", "xcomp", "advcl", "dative"}]
            )
            embedded_elements_count.append(n_embedded)
        return np.mean(embedded_elements_count)
    
    # putting it all together!
    def featurize(self, segments):
        feature_dicts = []
        docs = self.nlp.pipe(segments)
        for doc in docs:
            feature_dict = {
                "n_trump_mentions": self.get_n_word_mentions(doc)["n_trump_mentions"],
                "n_future_oriented_words": self.get_n_future_oriented_words(doc),
                "n_words_before_main_verb": self.get_n_words_before_main_verb(doc),
                "n_complex_clauses": self.get_n_complex_clauses(doc),
                "mean_sent_length": self.get_sent_length_stats(doc)["mean_sent_length"],
            }
            feature_dicts.append(feature_dict)
        return feature_dicts

In [16]:
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [17]:
#NLP preprocess /TBI / read again to apply new features
#https://www.analyticsvidhya.com/blog/2023/04/end-to-end-nlp-project-on-quora-duplicate-questions-pair-identification/

def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
    
    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)
        
    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()
    
    return q

In [18]:
def topic_modeling_ldamodel(data_df = None, text_col = 'utterance', coherence='u_mass', num_topics = 5,
        num_topics_plot_chk = True, extend_stopword_list=[], filter_extremes=False, visualize_topics = False):

    '''
    ver 1.0 May12,2023 initial version
    ver 1.1 May15,2023 added topic_name,filter_extremes
    
    Note
    coherence='u_mass' # C_v, C_p, C_uci, C_umass, C_npmi, C_a
    
    filter_extremes should be true when database ig high
    '''
    import pandas as pd
    import matplotlib.pyplot as plt
    import spacy

    #import en_core_web_md
    from gensim.corpora.dictionary import Dictionary
    from gensim.models import LdaMulticore
    from gensim.models import CoherenceModel

    series_data = data_df[text_col]

    nlp = spacy.load("en_core_web_sm")

    removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

    if extend_stopword_list:
        nlp.Defaults.stop_words |= set(extend_stopword_list)
        
    tokens = []
    for summary in nlp.pipe(series_data):
        proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
        tokens.append(proj_tok)

    data_df['tokens'] = tokens

    #3 Create dictionary and corpus 
    #Dictionary: The idea of the dictionary is to give each token a unique ID.
    #Corpus: Having assigned a unique ID to each token, the corpus simply contains each ID and its frequency 

    # I will apply the Dictionary Object from Gensim, which maps each word to their unique ID:
    dictionary = Dictionary(data_df['tokens'])
    print('dictionary:', dictionary.token2id)

    #filter out low-frequency and high-frequency tokens, also limit the vocabulary to a max of 1000 words:
    #dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

    corpus = [dictionary.doc2bow(doc) for doc in data_df['tokens']]
    #print('corpus:', corpus)

    # #4 Coherence score and perplexity provide a convinent way to measure how good a given topic model is.
    # #https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920
    # id2word = TBC

    # lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
    #                                            id2word=id2word,
    #                                            num_topics=20, 
    #                                            random_state=100,
    #                                            update_every=1,
    #                                            chunksize=100,
    #                                            passes=10,
    #                                            alpha='auto',
    #                                            per_word_topics=True)

    # #4.1 Compute Perplexity - a measure of how good the model is. lower the better.
    # print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

    # #4.2 Compute Coherence Score -Higher the topic coherence, the topic is more human interpretable.
    # coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    # coherence_lda = coherence_model_lda.get_coherence()
    # print('\nCoherence Score: ', coherence_lda)


    #4.3 coherence score - measures the degree of semantic similarity between high scoring words in each topic.
    #The coherence score for C_v ranges from 0 (complete incoherence) to 1 (complete coherence). Values above 0.5 are fairly good
    if num_topics_plot_chk:
        topics = []
        score = []
        for i in range(1,20,1):
            lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=10, num_topics=i, workers = 4, passes=10, random_state=100)
            cm = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
            topics.append(i)
            score.append(cm.get_coherence())
        _=plt.plot(topics, score)
        _=plt.xlabel('Number of Topics')
        _=plt.ylabel('Coherence Score')
        plt.show()

    # check best topic w.r.t score - ranges from 0 to 1; Values above 0.5 are fairly good
    topic_coherence_score_dict = {k: v for k, v in zip(topics, score)} #TBU/for ref
    num_topics = score.index(min(score)) if num_topics is None else num_topics

    #5 Model building - chg num_topics as per above plot
    # LdaMulticore, uses all CPU cores to parallelize and speed up model training.
    lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=num_topics, workers = 4, passes=10)
    #if does not work LdaMulticore then use an equivalent single-core model LdaModel
    #gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

    topic_proba_list = lda_model.print_topics(-1)

    # sort the list of tuples (topic, score) and add it
    data_df['topic_id'] = [sorted(lda_model[corpus][text], key = lambda x: x[1], reverse=True)[0][0] for text in range(len(data_df[text_col]))]
    data_df['topic_name'] = data_df['topic_id'].apply(lambda x: topic_proba_list[x][1].split('+')[0].split('*')[1].strip().strip('"'))
    
    # Visualize topics / installation issue
    if visualize_topics:
        lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
        pyLDAvis.display(lda_display)

        # Save the report
        pyLDAvis.save_html(lda_display, 'index.html')

    return topic_proba_list, data_df

In [19]:
def spacy_textacy_entity_extraction_knowlege_graph(text_data, ent_type = "DATE",
                records_groupby_id = None, ent_plot_filter = None, plot_save_path=None):

    '''
    1.1 May12,2023 added records_groupby_id option/ used to group records like chat @ user is level
    1.0 May03,2023 initial version

    plot_save_path: if not none then it will create Network Graph
    ent_type = NUMERIC, PERSON, ORG, LOC, NUMERIC, DATE
    '''

    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt

    import spacy 
    import textacy  #0.12.0
    import networkx as nx  #3.0 (also pygraphviz==1.10)
    import dateparser #1.1.7
    

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text_data)

    #3 from text to a list of sentences
    lst_docs = [sent for sent in doc.sents]

    dic_sub_vrb_obj = {"id":[], "text":[], "sub":[], "relation":[], "object":[]}
    dic_ent = {"id":[], "text":[], ent_type:[]}
    dic_all_ent = {"id":[], "text":[], 'all_ent':[]}
    dic_all_noun = {"id":[], "text":[], 'noun':[]}

    for n, sentence in enumerate(lst_docs):
        #print(f'\n sentence no:{n}, sentence:{sentence}')
        lst_sub_vrb_obj = list(textacy.extract.subject_verb_object_triples(sentence))
        lst_ent = list(textacy.extract.entities(sentence, include_types={ent_type}))
        lst_all_ent = list(textacy.extract.entities(sentence))
        lst_all_noun = list(textacy.extract.noun_chunks(sentence))
        list_all_tokens = list(textacy.extract.words(sentence))

        #1 lst_ent
        if len(lst_ent) > 0:
            for attr in lst_ent:
                dic_ent["id"].append(n)
                dic_ent["text"].append(sentence.text)
                dic_ent[ent_type].append(str(attr))
        else:
            dic_ent["id"].append(n)
            dic_ent["text"].append(sentence.text)
            dic_ent[ent_type].append(np.nan)

        dtf_ent = pd.DataFrame(dic_ent)

        #2 lst_all_ent
        if len(lst_all_ent) > 0:
            for attr1 in lst_all_ent:
                dic_all_ent["id"].append(n)
                dic_all_ent["text"].append(sentence.text)
                dic_all_ent['all_ent'].append(str(attr1))
        else:
            dic_all_ent["id"].append(n)
            dic_all_ent["text"].append(sentence.text)
            dic_all_ent['all_ent'].append(np.nan)

        dtf_all_ent = pd.DataFrame(dic_all_ent)
        if records_groupby_id:
            dtf_all_ent = dtf_all_ent.groupby(records_groupby_id).agg(lambda x: list(x)).reset_index()

        #3 lst_sub_vrb_obj
        if len(lst_sub_vrb_obj) > 0:
            for m, sent in enumerate(lst_sub_vrb_obj): #one sentence can have more than 1 one sub/verb/obj
                subj = "_".join(map(str, sent.subject))
                obj  = "_".join(map(str, sent.object))
                relation = "_".join(map(str, sent.verb))
                dic_sub_vrb_obj["id"].append(n)
                dic_sub_vrb_obj["text"].append(sentence.text)
                dic_sub_vrb_obj["sub"].append(subj)
                dic_sub_vrb_obj["object"].append(obj)
                dic_sub_vrb_obj["relation"].append(relation)
        else:
            dic_sub_vrb_obj["id"].append(n)
            dic_sub_vrb_obj["text"].append(sentence.text)
            dic_sub_vrb_obj["sub"].append(np.nan)
            dic_sub_vrb_obj["object"].append(np.nan)
            dic_sub_vrb_obj["relation"].append(np.nan)

        dtf_sub_vrb_obj = pd.DataFrame(dic_sub_vrb_obj)

        #4 lst_all_noun
        if len(lst_all_noun) > 0:
            for attr1 in lst_all_noun:
                dic_all_noun["id"].append(n)
                dic_all_noun["text"].append(sentence.text)
                dic_all_noun['noun'].append(str(attr1))
        else:
            dic_all_noun["id"].append(n)
            dic_all_noun["text"].append(sentence.text)
            dic_all_noun['all_ent'].append(np.nan)

        dtf_all_noun = pd.DataFrame(dic_all_noun)
        dtf_all_noun = dtf_all_noun.groupby('id').agg(lambda x: list(x)).reset_index()

    # merge the all files
    dtf_final = dtf_sub_vrb_obj.merge(dtf_ent[[ent_type, 'id']], how="left", on="id")
    dtf_final = dtf_final.merge(dtf_all_ent[['all_ent', 'id']], how="left", on="id")
    dtf_final = dtf_final.merge(dtf_all_noun[['noun', 'id']], how="left", on="id")

    if plot_save_path:
        if ent_plot_filter:
            dtf_final = dtf_final[(dtf_final["sub"]==ent_plot_filter) | (dtf_final["object"]==ent_plot_filter)]

        ## create full graph
        G = nx.from_pandas_edgelist(dtf_final, source="sub", target="object", 
                                    edge_attr="relation", create_using=nx.DiGraph())
        ## plot
        plt.figure(figsize=(15,10))

        pos = nx.spring_layout(G, k=1)
        node_color = "skyblue"
        edge_color = "black"

        nx.draw(G, pos=pos, with_labels=True, node_color=node_color, 
                edge_color=edge_color, cmap=plt.cm.Dark2, 
                node_size=2000, connectionstyle='arc3,rad=0.1')

        nx.draw_networkx_edge_labels(G, pos=pos, label_pos=0.5, 
                                 edge_labels=nx.get_edge_attributes(G,'relation'),
                                 font_size=12, font_color='black', alpha=0.6)
        plt.show()
        plt.savefig(plot_save_path+'EntityKnowledgeGraph.png')

    return dtf_final

In [20]:
def nlp_clustering(data_df, text_col):
    '''
    May 15,2023: initial version
    '''
    import pandas as pd
    import numpy as np
    from sklearn.cluster import MiniBatchKMeans
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt

    random_state = 0 

    vec = TfidfVectorizer(stop_words="english")
    vec.fit(data_df[text_col].values)
    features = vec.transform(data_df[text_col].values)

    cls = MiniBatchKMeans(n_clusters=5, random_state=random_state)
    cls.fit(features)
    
    y_pred= cls.predict(features)
    
    data_df['cluster_label'] = cls.labels_
    return data_df